In [1]:
import os
import re
import json
import base64
import tiktoken
import time
import pandas as pd
import openai
from tqdm.notebook import tqdm
from openai import OpenAI


In [2]:
MODEL_LIMITS = {
    "gpt-3.5-turbo-0125": 16_385,
    "gpt-4-turbo-2024-04-09": 128_000,
    "gpt-4o-2024-05-13": 128_000
}

# The cost per token for each model input.
MODEL_COST_PER_INPUT = {
    "gpt-3.5-turbo-0125": 0.0000005,
    "gpt-4-turbo-2024-04-09": 0.00001,
    "gpt-4o-2024-05-13": 0.000005
}

# The cost per token for each model output.
MODEL_COST_PER_OUTPUT = {
    "gpt-3.5-turbo-0125": 0.0000015,
    "gpt-4-turbo-2024-04-09": 0.00003,
    "gpt-4o-2024-05-13": 0.000015
}



In [3]:
# If the question is a multi-choice question and you are unsure which one is correct, you must guess an option.  Please don't ask me any questions and give me the answer in the response.

def get_gpt_res(text, image, model):
    if image and model[:7] != "gpt-3.5":
        base64_image = encode_image(image)
        image_code = {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"}
        }
    
        response = client.chat.completions.create(
          model=model,
          messages=[
            {
              "role": "system",
              "content": [
                {
                  "type": "text",
                  "text": "You are a data analyst. I will give  you a background introduction and data analysis question. You must answer the question.  "
                }
              ]
            },
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": text
                },
                image_code
              ]
            }
          ],
          temperature=0,
          max_tokens=2256,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
    else:
        response = client.chat.completions.create(
          model=model,
          messages=[
            {
              "role": "system",
              "content": [
                {
                  "type": "text",
                  "text": "You are a data analyst. I will give  you a background introduction and data analysis question. You must answer the question. "
                }
              ]
            },
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": text
                }
              ]
            }
          ],
          temperature=0,
          # max_tokens=2256,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
    return response

In [4]:
samples = []
with open("./data.json", "r") as f:
    for line in f:
        samples.append(eval(line.strip()))


In [5]:
def gpt_tokenize(string: str, encoding) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = len(encoding.encode(string))
    return num_tokens

def find_jpg_files(directory):
    jpg_files = [file for file in os.listdir(directory) if file.lower().endswith('.jpg') or file.lower().endswith('.png')]
    return jpg_files if jpg_files else None

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


def find_excel_files(directory):
    jpg_files = [file for file in os.listdir(directory) if (file.lower().endswith('xlsx') or file.lower().endswith('xlsb') or file.lower().endswith('xlsm')) and not "answer" in file.lower()]
    return jpg_files if jpg_files else None

def read_excel(file_path):
    # 读取Excel文件中的所有sheet
    xls = pd.ExcelFile(file_path)
    sheets = {}
    for sheet_name in xls.sheet_names:
        sheets[sheet_name] = xls.parse(sheet_name)
    return sheets

def dataframe_to_text(df):
    # 将DataFrame转换为文本
    text = df.to_string(index=False)
    return text

def combine_sheets_text(sheets):
    # 将所有sheet的文本内容组合起来
    combined_text = ""
    for sheet_name, df in sheets.items():
        sheet_text = dataframe_to_text(df)
        combined_text += f"Sheet name: {sheet_name}\n{sheet_text}\n\n"
    return combined_text

def read_txt(path):
    with open(path, "r") as f:
        return f.read()

def truncate_text(text, max_tokens=128000):
    # 计算当前文本的token数
    tokens = text.split()
    if len(tokens) > max_tokens:
        # 截断文本以确保不超过最大token数
        text = ' '.join(tokens[-max_tokens:])
    return text

In [8]:
client = OpenAI(api_key="your-api-key")

tokens4generation = 6000
# model = "gpt-3.5-turbo-0125"
model = "gpt-4-turbo-2024-04-09"
data_path = "./data/"
total_cost = 0
encoding = tiktoken.encoding_for_model(model)
for id in tqdm(range(len(samples))):
    # print(sample)
    sample =samples[id]
    if len(sample["questions"]) > 0:
        start = sample["questions"][0]
        end = sample["questions"][-1]
        # print(start)
        # print(end)
        image = find_jpg_files(os.path.join(data_path, sample["id"]))
        if image:
            image = os.path.join(data_path, sample["id"], image[0])
        
        excel_content = ""
        excels = find_excel_files(os.path.join(data_path, sample["id"]))
        if excels:
            
            for excel in excels:
                excel_file_path = os.path.join(data_path,  sample["id"], excel)
                # print(excel_file_path)
                sheets = read_excel(excel_file_path)
                combined_text = combine_sheets_text(sheets)
                excel_content += f"The excel file {excel} is: " + combined_text

        introduction = read_txt(os.path.join(data_path, sample["id"], "introduction.txt"))
        questions = []
        for question_name in sample["questions"]:
            questions.append(read_txt(os.path.join(data_path, sample["id"], question_name+".txt")))
            
        # print(workbooks)
        
        text = ""
        if excel_content:
            text += f"The workbook is detailed as follows. {excel_content} \n"
        text += f"The introduction is detailed as follows. \n {introduction} \n"
        answers = []
        for question in questions:
            prompt = text +  f"The questions are detailed as follows. \n {question}"
        
            # print(len(encoding.encode(prompt)))
            cut_text = encoding.decode(encoding.encode(prompt)[tokens4generation-MODEL_LIMITS[model]:])
            # print(len(encoding.encode(prompt)))
            # print(prompt)
        # text = truncate_text(text, 20000)
            start = time.time()
            response = get_gpt_res(cut_text, image, model)
            cost = response.usage.completion_tokens * MODEL_COST_PER_OUTPUT[model] + response.usage.prompt_tokens * MODEL_COST_PER_INPUT[model]
            
            answers.append({"id": sample["id"], "model": response.model, "input": response.usage.prompt_tokens,
                            "output": response.usage.completion_tokens, "cost": cost, "time": time.time()-start, "response": response.choices[0].message.content})
            total_cost += cost
            print("Total cost: ", total_cost)
            # break
        save_path = os.path.join("./save_process", model)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        with open(os.path.join(save_path, sample['id']+".json"), "w") as f:
            for answer in answers:
                json.dump(answer, f)
                f.write("\n")


  0%|          | 0/35 [00:00<?, ?it/s]

Total cost:  36.999739999999996
Total cost:  38.250449999999994
Total cost:  39.498639999999995
Total cost:  40.747339999999994
Total cost:  41.992529999999995
Total cost:  43.23591999999999
Total cost:  44.484019999999994
Total cost:  45.72899999999999
Total cost:  46.97565999999999
Total cost:  48.22759999999999
Total cost:  49.47311999999999
Total cost:  50.719599999999986
Total cost:  51.96382999999999
Total cost:  53.212769999999985
Total cost:  54.45198999999999
Total cost:  54.486009999999986
Total cost:  54.51985999999999
Total cost:  54.55831999999999
Total cost:  54.593969999999985
Total cost:  54.621729999999985
Total cost:  54.65634999999998
Total cost:  54.690989999999985
Total cost:  54.728549999999984
Total cost:  54.76124999999998
Total cost:  54.792459999999984
Total cost:  54.82964999999999
Total cost:  54.867389999999986
Total cost:  54.90312999999998
Total cost:  54.93681999999998
Total cost:  54.969619999999985
Total cost:  55.007139999999985
Total cost:  55.040789

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}